In [1]:
from ltgee import LandTrendr, LandsatComposite, LtCollection
from datetime import date
import geemap
import ee

# Authenticate with Google Earth Engine before using. See link below for more information:
# https://developers.google.com/earth-engine/guides/auth

# add in your own project name
ee.Initialize()

In [2]:
# Example arguments to pass to the LandTrendr class. See docstring of LandTrendr for more information'
composite_params = {
    "start_date": date(1985, 6,1),
    "end_date": date(2017, 9,1),
    "area_of_interest": ee.Geometry({
        'type': 'Polygon',
        'coordinates': [
            [
                [-122.37202331327023,44.62585686599272],
                [-122.26765319608273,44.62585686599272],
                [-122.26765319608273,44.696185837887384],
                [-122.37202331327023,44.696185837887384],
                [-122.37202331327023,44.62585686599272],
                ]
            ]
    }),
    "mask_labels": ['cloud', 'shadow', 'snow', 'water'],
    "debug": True
}
lt_collection_params = {
        "sr_collection": LandsatComposite(**composite_params),
        # "sr_collection": composite_params, # - you may also just pass in your own collection or the params directly
        "index": 'NBR',
        "ftv_list": ['TCB', 'TCG', 'TCW', 'NBR'],
}
lt_params = {
    "lt_collection": LtCollection(**lt_collection_params),
    # "lt_collection": lt_collection_params, # - you may also just pass in your own collection or the params directly
    "run_params": {
            "maxSegments": 6,
            "spikeThreshold": 0.9,
            "vertexCountOvershoot":  3,
            "preventOneYearRecovery":  True,
            "recoveryThreshold":  0.25,
            "pvalThreshold":  .05,
            "bestModelProportion":  0.75,
            "minObservationsNeeded": 6,
        }
}

In [3]:
# Instantiating LandTrendr object. Note: the LandTrendr algorithm will be run greedily on GEE (ie the script will immediately request to run the algorithm on GEE as soon as it has all the collections built unless lt.run=False).
lt = LandTrendr(**lt_params)
lt

In [4]:
# You may access the collections using the properties below
lt.lt_collection.sr_collection, lt.lt_collection, lt.lt_collection.sr_collection.clear_pixel_count_collection

(<ltgee.landtrendr.LandsatComposite at 0x1a80299b910>,
 <ee.imagecollection.ImageCollection at 0x1a807be4040>)

In [5]:
# Example 1: Displaying images from Landtrendr

lt_tcb = lt.select('ftv_nbr_fit').clip(composite_params['area_of_interest'])
map = geemap.Map()
map.centerObject(composite_params['area_of_interest'], 13)
map.addLayer(lt_tcb, {}, 'lt_nbr')
map.addLayer(composite_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [6]:
# Example 2: Displaying collection to band stack

# Note: lt_collection outside of the class can be passed in
collection_to_band_stack = lt.collection_to_band_stack(lt.lt_collection, start_date=composite_params["start_date"],  end_date=composite_params["end_date"])

map = geemap.Map()
map.centerObject(composite_params['area_of_interest'], 13)
map.addLayer(collection_to_band_stack, {"bands":["2000"],"min":-100,"max":1000,"palette":["ff2f0d","fff825","0ab308"]})
map.addLayer(composite_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Example 3: Displaying change data
# Note: Change params does not require all keys to be present and will only process what is passed in.

change_params = {
                    'delta': 'loss',
                    'sort': 'greatest',
                    'years': {'start': composite_params["start_date"].year + 1, 'end': composite_params["end_date"].year},
                    'mag': {'value': 200, 'operator': '>' },
                    'dur': {'value': 4, 'operator': '<'},
                    'preval': {'value': 300, 'operator': '>'},
                    'mmu': {'value': 5}
                }
change_image = lt.get_change_map(change_params)

palette = ['#9400D3', '#4B0082', '#0000FF', '#00FF00', '#FFFF00', '#FF7F00', '#FF0000']
yod_vis_params = {
  'min': change_params['years']['start'],
  'max': change_params['years']['end'],
  'palette': palette
}
mag_vis_params = {
  'min': 200,
  'max': 800,
  'palette': palette
}


map = geemap.Map()
map.centerObject(composite_params['area_of_interest'], 13)
map.addLayer(change_image.select(['mag']), yod_vis_params, 'Magnitude of Change')
map.addLayer(change_image.select(['yod']), mag_vis_params, 'Year of Detection')
map.addLayer(composite_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [8]:
# Example 4: Displaying segment data
# In the top right corner hover over the wrench and click the 'i' to use the inspector. Click any point in the aoi to see values.

seg_info = lt.get_segment_data('all', index_flip=True).clip(composite_params['area_of_interest'])
map = geemap.Map()
map.centerObject(composite_params['area_of_interest'], 13)
map.addLayer(seg_info, {}, 'seg_info')
map.addLayer(composite_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [9]:
# Example 5: Displaying rgb image

vis_params = {
  'min': [604, -49, -2245],
  'max': [5592, 3147, 843],
  'gamma': [1, 1, 1]
}

lt_rgb = lt.get_fitted_rgb_col(['TCB', 'TCG', 'TCW'], vis_params, composite_params["start_date"], composite_params["end_date"])
map = geemap.Map()
map.centerObject(composite_params['area_of_interest'], 13)
map.addLayer(lt_rgb, {}, 'lt_rgb')
map.addLayer(composite_params['area_of_interest'], {}, 'area_of_interest')
map

Map(center=[44.661026124368775, -122.31983825467653], controls=(WidgetControl(options=['position', 'transparen…

In [10]:
# Example 6: Creating a gif using data from landtrendr

vid_params = {
  'crs': 'EPSG:3857',
  'dimensions': 300,
  'region': composite_params['area_of_interest'],
  'framesPerSecond': 8
}
lt_rgb.getVideoThumbURL(vid_params)

'https://earthengine.googleapis.com/v1/projects/ee-myscon/videoThumbnails/e0dc8e1f5c12fd0b585a3722d342574a-dac57f3a4651957394c53b1ff8407a84:getPixels'